## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-05-21-50-20 +0000,nypost,Here’s how much the Powerball winner will owe ...,https://nypost.com/2025/09/05/us-news/heres-ho...
1,2025-09-05-21-46-13 +0000,nyt,G20 Summit in 2026 to Be Held at Trump’s Club ...,https://www.nytimes.com/2025/09/05/us/politics...
2,2025-09-05-21-45-17 +0000,nypost,Adams defiantly insists he’s staying in NYC ma...,https://nypost.com/2025/09/05/us-news/adams-de...
3,2025-09-05-21-44-09 +0000,nyt,South Koreans Are Swept Up in Immigration Raid...,https://www.nytimes.com/2025/09/05/us/georgia-...
4,2025-09-05-21-41-52 +0000,nyt,Israel Steps Up Attacks on Gaza City Ahead of ...,https://www.nytimes.com/2025/09/05/world/middl...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2370/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
10,trump,59
32,up,13
217,new,12
3,will,11
446,rayner,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
310,2025-09-05-01-00-00 +0000,wsj,President Trump’s sprint to take control of th...,https://www.wsj.com/politics/policy/trump-fed-...,116
8,2025-09-05-21-37-00 +0000,wsj,President Trump signed an executive order to r...,https://www.wsj.com/politics/national-security...,110
296,2025-09-05-02-44-00 +0000,wsj,President Trump was within his authority when ...,https://www.wsj.com/politics/national-security...,108
304,2025-09-05-01-45-00 +0000,wsj,President Trump on Thursday led leaders of the...,https://www.wsj.com/politics/trump-tech-ceo-ro...,97
187,2025-09-05-13-39-23 +0000,nypost,US deploying 10 fighter jets to Puerto Rico in...,https://nypost.com/2025/09/05/us-news/us-deplo...,96


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
310,116,2025-09-05-01-00-00 +0000,wsj,President Trump’s sprint to take control of th...,https://www.wsj.com/politics/policy/trump-fed-...
56,57,2025-09-05-20-08-47 +0000,nyt,Elon Musk Could Become First Trillionaire Unde...,https://www.nytimes.com/2025/09/05/business/el...
153,57,2025-09-05-15-41-00 +0000,wsj,The Florida immigration detention facility kno...,https://www.wsj.com/politics/policy/florida-al...
31,47,2025-09-05-20-58-03 +0000,nypost,Top secret SEAL Team 6 killed North Korean civ...,https://nypost.com/2025/09/05/us-news/top-secr...
162,45,2025-09-05-15-18-34 +0000,nyt,"After Boat Strike, Rubio Says U.S. Will Help O...",https://www.nytimes.com/2025/09/04/world/ameri...
8,45,2025-09-05-21-37-00 +0000,wsj,President Trump signed an executive order to r...,https://www.wsj.com/politics/national-security...
268,44,2025-09-05-07-53-33 +0000,wsj,Stock Market Today: Weak Jobs Data Boosts Rate...,https://www.wsj.com/livecoverage/jobs-report-a...
241,43,2025-09-05-09-42-52 +0000,nypost,Putin warns Western troops deployed to Ukraine...,https://nypost.com/2025/09/05/world-news/russi...
2,40,2025-09-05-21-45-17 +0000,nypost,Adams defiantly insists he’s staying in NYC ma...,https://nypost.com/2025/09/05/us-news/adams-de...
155,40,2025-09-05-15-39-10 +0000,nyt,"Angela Rayner, UK Deputy Prime Minister, Resig...",https://www.nytimes.com/2025/09/05/world/europ...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
